# **Capítulo 3: Fontes de Dados Financeiros com Python**

Neste capítulo, vamos aprender a usar bibliotecas Python especializadas para buscar dados financeiros diretamente de fontes online, como cotações de ações, moedas e indicadores econômicos. Isso elimina a necessidade de baixar arquivos manualmente.

## **3.1 Bibliotecas para Dados Financeiros**

Existem diversas bibliotecas em Python para se conectar a fontes de dados (APIs). As duas que usaremos como principais exemplos são:

1.  **`yfinance`**: Uma biblioteca extremamente popular para baixar dados de cotações de ações, ETFs e índices do Yahoo Finance.
2.  **`python-bcb`**: Para obter séries temporais de dados econômicos brasileiros diretamente do Sistema Gerenciador de Séries Temporais (SGS) do Banco Central do Brasil.

### **3.1.1 yfinance: Cotações de Ações e Índices**

O `yfinance` é a ferramenta ideal para obter dados de ativos negociados em bolsa. Para ações brasileiras, usamos o mesmo código do home broker, adicionando o sufixo `.SA` (ex: `PETR4.SA`). A principal função que usaremos é a `yf.download()`.

Primeiro, você precisa instalar a biblioteca no seu terminal.

**No seu terminal (apenas uma vez):**
```bash

In [ ]:
pip install yfinance

Um passo fundamental após carregar um conjunto de dados é inspecioná-lo para garantir que tudo está correto. Vamos dar uma olhada nas primeiras e últimas linhas do nosso DataFrame da PETR4.

Para isso, usamos dois métodos muito úteis do Pandas:
* `.head()`: Exibe as 5 primeiras linhas.
* `.tail()`: Exibe as 5 últimas linhas.

In [ ]:
import yfinance as yf

# Baixando dados diários da Petrobras (PETR4) para o ano de 2023
# A função retorna um DataFrame do Pandas
petr4 = yf.download('PETR4.SA', start='2023-01-01', end='2023-12-31', auto_adjust=False)

# Exibindo as 5 primeiras linhas
print("--- Dados da PETR4 ---")
print(petr4.head())

# Exibindo as 5 últimas linhas
print("\n--- Fim dos Dados da PETR4 ---")
print(petr4.tail())

A função retorna um DataFrame com as seguintes colunas: `Open` (abertura), `High` (máxima), `Low` (mínima), `Close` (fechamento), `Adj Close` (fechamento ajustado para proventos) e `Volume`.

Também podemos baixar dados de múltiplos ativos de uma só vez.

In [ ]:
import yfinance as yf

# Definindo os tickers e o período
tickers = ['ITSA4.SA', 'VALE3.SA', '^BVSP']
start_date = '2023-01-01'
end_date = '2023-12-31'

# 1. Baixamos o DataFrame completo, com todos os dados (Open, High, Low, Close, etc.)
dados_completos = yf.download(tickers, start=start_date, end=end_date, auto_adjust=False)

# 2. (Opcional) Vamos inspecionar a estrutura das colunas para entender
print("--- Estrutura das Colunas (MultiIndex) ---")
print(dados_completos.columns)

# 3. Agora, selecionamos o nível superior 'Adj Close'. Isso nos dará um DataFrame
#    apenas com os preços de fechamento ajustado de todos os tickers.
dados_adj_close = dados_completos['Adj Close']
dados_close = dados_completos['Close']

# 4. Exibindo as 5 primeiras linhas do resultado correto
print("\n--- DataFrame Apenas com 'Adj Close' ---")
print(dados_adj_close.head())
print("\n--- DataFrame Apenas com 'Close' ---")
print(dados_close.head())

## **3.1.2 python-bcb: Dados Econômicos do Banco Central do Brasil**

Para obter dados da economia brasileira, como a taxa Selic, IPCA, IGP-M ou a cotação do dólar, usamos a biblioteca `python-bcb`. Ela se conecta diretamente ao **SGS (Sistema Gerenciador de Séries Temporais)** do Banco Central do Brasil.

Primeiro, você precisa instalar a biblioteca no seu terminal.

**No seu terminal (apenas uma vez):**
```bash

In [ ]:
pip install python-bcb

Cada série de dados no Banco Central tem um código único. Para encontrar o código de uma série, podemos usar a função `sgs.search_sgs()`. Depois, usamos o código encontrado na função `sgs.get()` para baixar os dados.

In [ ]:
from bcb import sgs

# Vamos usar o código 11, que sabemos ser da taxa Selic diária.
# Passamos um dicionário onde a chave é o nome que queremos para a coluna ('Selic') 
# e o valor é o código (11).
selic = sgs.get({'Selic': 11}, start='2023-01-01')

print("--- Histórico da Taxa Selic Diária ---")
print(selic.tail()) # Usando .tail() para ver os dados mais recentes

In [ ]:
# Baixando os últimos 12 meses do IPCA (código 433) e do IGP-M (código 189)
inflacao = sgs.get({'IPCA': 433, 'IGP-M': 189}, last=12)

print("\n--- Últimos 12 meses de Inflação (IPCA vs IGP-M) ---")
print(inflacao.head())

A busca nos mostra que a taxa Selic diária tem o código **11**. Agora, vamos usar esse código para baixar os dados históricos.

In [ ]:
# Baixando a série da Selic a partir de 01/01/2023
# Passamos um dicionário onde a chave é o nome que queremos para a coluna e o valor é o código
selic = sgs.get({'Selic': 11}, start='2023-01-01')

print("--- Histórico da Taxa Selic Diária ---")
print(selic.head())

Também podemos baixar múltiplas séries de uma vez, passando um dicionário com vários códigos.

In [ ]:
# Baixando os últimos 12 meses do IPCA (código 433) e do IGP-M (código 189)
inflacao = sgs.get({'IPCA': 433, 'IGP-M': 189}, last=12)

print("--- Últimos 12 meses de Inflação (IPCA vs IGP-M) ---")
print(inflacao.head())

## **3.2 Manipulação de Séries Temporais com Pandas**

Tanto o `yfinance` quanto o `python-bcb` retornam DataFrames do Pandas, que são perfeitos para análise de séries temporais. A principal característica que torna isso possível é o **`DatetimeIndex`**, um tipo especial de índice composto por datas. Ter as datas como índice nos permite agrupar, fatiar e manipular os dados com base no tempo de forma muito eficiente.

In [ ]:
import yfinance as yf

# Vamos baixar um ativo de exemplo para esta seção
itsa4 = yf.download('ITSA4.SA', start='2022-01-01', end='2023-12-31')

# Vamos verificar o tipo do índice do nosso DataFrame
print(f"O tipo do índice é: {type(itsa4.index)}")

print("\nVisualizando os dados:")
print(itsa4.head())

Um DataFrame com `DatetimeIndex` tem dois componentes principais que podemos acessar separadamente:
* **`.index`**: O objeto que contém todas as datas (o índice).
* **`.values`**: Um array NumPy contendo todos os valores numéricos do DataFrame (os dados em si).

In [ ]:
# Acessando apenas o índice (as datas)
print("--- Objeto de Índice (Datas) ---")
print(itsa4.index)


# Acessando apenas os valores (como uma matriz)
print("\n--- Array de Valores ---")
print(itsa4.values)

### Convertendo a Periodicidade (Resample)

Uma das tarefas mais comuns é mudar a frequência dos dados (ex: de diário para mensal). No Pandas, fazemos isso com o método `.resample()`. Ele agrupa os dados por uma nova frequência e depois aplicamos uma função de agregação (como `.last()`, `.first()`, `.sum()`, `.mean()`) para definir como os dados agrupados devem ser representados.

Usamos códigos para definir a frequência:
* `M`: Fim do Mês (`Month End`)
* `MS`: Início do Mês (`Month Start`)
* `Q`: Fim do Trimestre (`Quarter End`)
* `Y`: Fim do Ano (`Year End`)

In [45]:
# Resample para dados mensais, pegando o último preço de cada mês
itsa4_mensal = itsa4.resample('ME').last()
print("--- Dados Mensais (Último dia do Mês) ---")
print(itsa4_mensal.head())

# Resample para dados trimestrais, calculando o preço médio de cada trimestre
itsa4_trimestral = itsa4.resample('QE').mean()
print("\n--- Dados Trimestrais (Média do Trimestre) ---")
print(itsa4_trimestral.head())

--- Dados Mensais (Último dia do Mês) ---
Price          Close      High       Low      Open    Volume
Ticker      ITSA4.SA  ITSA4.SA  ITSA4.SA  ITSA4.SA  ITSA4.SA
Date                                                        
2022-01-31  6.354131  6.397781  6.235653  6.266832  33804672
2022-02-28  6.248127  6.254362  6.135885  6.160827  34463544
2022-03-31  6.773339  6.880452  6.767038  6.842648  16056397
2022-04-30  5.803020  6.017246  5.796719  5.929035  40665594
2022-05-31  6.042449  6.073953  5.960538  5.985742  48779592

--- Dados Trimestrais (Média do Trimestre) ---
Price          Close      High       Low      Open        Volume
Ticker      ITSA4.SA  ITSA4.SA  ITSA4.SA  ITSA4.SA      ITSA4.SA
Date                                                            
2022-03-31  6.233499  6.294401  6.156407  6.218140  3.225892e+07
2022-06-30  5.905954  5.979601  5.856837  5.934849  2.883327e+07
2022-09-30  5.732475  5.782818  5.667512  5.716477  2.436769e+07
2022-12-31  6.324385  6.409759  

### Movendo o Índice de Data para uma Coluna

Às vezes, é útil ter a data como uma coluna normal do DataFrame em vez de ser o índice. Para fazer isso, usamos o método `.reset_index()`.

In [52]:
# Criando uma cópia para não alterar nosso DataFrame original
itsa4_com_coluna_data = itsa4.reset_index()

print("--- DataFrame com a data como coluna ---")
print(itsa4_com_coluna_data.head())

--- DataFrame com a data como coluna ---
Price        Date     Close      High       Low      Open    Volume
Ticker             ITSA4.SA  ITSA4.SA  ITSA4.SA  ITSA4.SA  ITSA4.SA
0      2022-01-03  5.547342  5.651893  5.485841  5.547342  34761437
1      2022-01-04  5.590392  5.639592  5.516591  5.553491  47291709
2      2022-01-05  5.528892  5.602693  5.485842  5.565793  42275995
3      2022-01-06  5.578093  5.621143  5.498142  5.522743  30914401
4      2022-01-07  5.645743  5.651893  5.541192  5.578092  26119001
